In [16]:
import tensorflow as tf
from tensorflow import keras
import seaborn as sns
import numpy as np
import matplotlib.pyplot as plt
import generate_inputs as gi
import file_methods as fm
import pandas as pd

print(tf.__version__)

2.2.0


In [19]:
# Import a template CT input file and iterate on it to make a test set, training set, and validation set.
template = gi.import_template('crunchModel/boxModel.in')

train_set = gi.generate_data_set(template, 'Seawater', 500, 0.0000000001, 0.00001, 'train')
test_set = gi.generate_data_set(template, 'Seawater', 50, 0.00001, 0.0001, 'test')
val_set = gi.generate_data_set(template, 'Seawater', 50, 0.00001, 0.0001, 'val')

The keyword "ION_EXCHANGE" you searched for does not exist. If you are sure that this keyword is in your input file, check your spelling.
The keyword "SURFACE_COMPLEXATION" you searched for does not exist. If you are sure that this keyword is in your input file, check your spelling.
The keyword "TRANSPORT" you searched for does not exist. If you are sure that this keyword is in your input file, check your spelling.
The keyword "FLOW" you searched for does not exist. If you are sure that this keyword is in your input file, check your spelling.
The keyword "PEST" you searched for does not exist. If you are sure that this keyword is in your input file, check your spelling.
The keyword "EROSION/BURIAL" you searched for does not exist. If you are sure that this keyword is in your input file, check your spelling.


In [20]:
fm.pickle_data_set(train_set, 'train')
fm.pickle_data_set(test_set, 'test')
fm.pickle_data_set(val_set, 'val')

In [29]:
# Put data into a dataframe for visualisation.
vis = pd.DataFrame()

for i in train_set:
    append = pd.DataFrame.from_dict(train_set[i].condition_blocks['Seawater'].primary_species)
    vis = vis.append(append, ignore_index = True)
    
# Create series from output data (pH).
pH = pd.DataFrame()

for i in train_set:
    pH = pH.append(train_set[i].results.results_dict['pH']['pH'], ignore_index = True)

pH.columns = ['pH']

# Normalise the pH to be on the scale 0-1ish
pH['pH'] = pH['pH'] / 14

x_train = vis.to_numpy()
y_train = pH.to_numpy()

print(np.shape(x_train))

(500, 20)


In [37]:
# Put data into a dataframe for visualisation.
vis = pd.DataFrame()

for i in test_set:
    append = pd.DataFrame.from_dict(test_set[i].condition_blocks['Seawater'].primary_species)
    vis = vis.append(append, ignore_index = True)
    
# Create series from output data (pH).
pH = pd.DataFrame()

for i in test_set:
    pH = pH.append(test_set[i].results.results_dict['pH']['pH'], ignore_index = True)

pH.columns = ['pH']

# Normalise the pH to be on the scale 0-1ish
pH['pH'] = pH['pH'] / 14

x_test = vis.to_numpy()
y_test = pH.to_numpy()

(50, 20)


In [33]:
pH.describe()

,pH
count,500.000000
mean,0.578219
std,0.004408
min,0.552807
25%,0.576846
50%,0.579630
75%,0.581268
max,0.582583


In [51]:
# Define the model geometry.
model = tf.keras.models.Sequential([
  #tf.keras.layers.Flatten(input_shape=(28, 28)),
  tf.keras.layers.Dense(128, activation='relu'),
  tf.keras.layers.Dropout(0.2),
  tf.keras.layers.Dense(10)
])

# Define the loss function.
mse = tf.keras.losses.MeanSquaredError()

# Compile the model.

model.compile(optimizer='adam',
              loss=mse,
              metrics=['accuracy', 'mse'])

In [52]:
model.fit(x_train, y_train, epochs=2000)

Epoch 1/2000
2/2 [==============================] - 0s 1ms/step - loss: 0.3312 - accuracy: 0.0000e+00 - mse: 0.3312
Epoch 2/2000
2/2 [==============================] - 0s 990us/step - loss: 0.3240 - accuracy: 0.0000e+00 - mse: 0.3240
Epoch 3/2000
2/2 [==============================] - 0s 888us/step - loss: 0.3162 - accuracy: 0.0000e+00 - mse: 0.3162
Epoch 4/2000
2/2 [==============================] - 0s 926us/step - loss: 0.3076 - accuracy: 0.0000e+00 - mse: 0.3076
Epoch 5/2000
2/2 [==============================] - 0s 867us/step - loss: 0.2985 - accuracy: 0.0000e+00 - mse: 0.2985
Epoch 6/2000
2/2 [==============================] - 0s 1ms/step - loss: 0.2896 - accuracy: 0.0000e+00 - mse: 0.2896
Epoch 7/2000
2/2 [==============================] - 0s 892us/step - loss: 0.2792 - accuracy: 0.0000e+00 - mse: 0.2792
Epoch 8/2000
2/2 [==============================] - 0s 1ms/step - loss: 0.2684 - accuracy: 0.0000e+00 - mse: 0.2684
Epoch 9/2000
2/2 [==============================] - 0s 1ms/ste

In [48]:
model.evaluate(x_test, y_test)

2/2 [==============================] - 0s 2ms/step - loss: 2.3809e-05 - accuracy: 0.0000e+00 - mse: 2.3809e-05


[2.3809085178072564e-05, 0.0, 2.3809085178072564e-05]